In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision ## Contains some utilities for working with the image data
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
#%matplotlib inline
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [3]:
dataset = MNIST(root = 'data/', download = True)
print(len(dataset))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16153366.06it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 494685.81it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4476365.95it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8183216.82it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

60000


In [4]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   t10k-labels-idx1-ubyte.gz   train-labels-idx1-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte     train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte	   train-images-idx3-ubyte.gz


In [5]:
import torchvision.transforms as transforms

train_data = MNIST(root='data/', train=True, download=True, transform=transforms.ToTensor())
test_data = MNIST(root='data/', train=False, download=True, transform=transforms.ToTensor())

In [6]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=10, shuffle=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([10, 1, 28, 28])
Shape of y: torch.Size([10]) torch.int64


In [11]:
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        return self.layers(x)

model = MLP().to(device)
print(model)

Using cpu device
MLP(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [15]:
def train(data_loader, model, criterion, optimizer):
    model.train()

    num_batches = len(data_loader)
    num_items = len(data_loader.dataset)

    total_loss = 0
    total_correct = 0
    for data, target in data_loader:
        # Copy data and targets to GPU
        data = data.to(device)
        target = target.to(device)

        # Do a forward pass
        output = model(data)

        # Calculate the loss



        loss = criterion(output, target)
        total_loss += loss

        # Count number of correct digits


        total_correct += (torch.argmax(output,dim = 1)== target).sum().item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_loss = total_loss/num_batches
    train_accuracy = total_correct/num_items

    print(f"Average loss: {train_loss:7f}, accuracy: {train_accuracy:.2%}")



In [16]:
def test(data_loader, model, criterion, optimizer):
    model.eval()

    num_batches = len(data_loader)
    num_items = len(data_loader.dataset)

    total_loss = 0
    total_correct = 0
    for data, target in data_loader:
        # Copy data and targets to GPU
        data = data.to(device)
        target = target.to(device)

        # Do a forward pass
        output = model(data)

        # Calculate the loss



        loss = criterion(output, target)
        total_loss += loss

        # Count number of correct digits


        total_correct += (torch.argmax(output,dim = 1)== target).sum().item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    test_loss = total_loss/num_batches
    test_accuracy = total_correct/num_items

    print(f"Average loss: {test_loss:7f}, accuracy: {test_accuracy:.2%}")



In [18]:
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_dataloader, model, criterion, optimizer)
    test(test_dataloader, model, criterion, optimizer)

Training epoch: 1
Average loss: 0.056357, accuracy: 98.30%
Average loss: 0.137443, accuracy: 96.02%
Training epoch: 2
Average loss: 0.059196, accuracy: 98.23%
Average loss: 0.088630, accuracy: 97.47%
Training epoch: 3
Average loss: 0.050689, accuracy: 98.50%
Average loss: 0.063414, accuracy: 98.30%
Training epoch: 4
Average loss: 0.043259, accuracy: 98.64%
Average loss: 0.057642, accuracy: 98.57%
Training epoch: 5
Average loss: 0.039762, accuracy: 98.81%
Average loss: 0.044697, accuracy: 98.70%
Training epoch: 6
Average loss: 0.034327, accuracy: 98.96%
Average loss: 0.052697, accuracy: 98.67%
Training epoch: 7
Average loss: 0.036392, accuracy: 98.94%
Average loss: 0.029222, accuracy: 99.26%
Training epoch: 8
Average loss: 0.031707, accuracy: 99.06%
Average loss: 0.037534, accuracy: 99.12%
Training epoch: 9
Average loss: 0.031991, accuracy: 99.09%
Average loss: 0.018975, accuracy: 99.52%
Training epoch: 10
Average loss: 0.024227, accuracy: 99.29%
Average loss: 0.043790, accuracy: 99.25%